<a href="https://colab.research.google.com/github/samarth1864/Calimeros_Research_Studio/blob/main/Calimeros_Research_Studio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
Calimeros Research Studio

### Mapping the Invisible - Prediction of unknown datapoints through Random Forest Decision tree

To predict unknown climatic data points across the city, a Random Forest Decision Tree method is employed, leveraging its accuracy and robustness in handling complex datasets. This approach uses existing IoT sensor data and applies feature engineering to enrich the dataset. By triangulating data with these parameters, the model ensures precise predictions for unmonitored locations, enabling accurate insights for the identified locations.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.cluster import KMeans

file_path = '/content/Existing_IOT_Barcelona (1).xlsx'
data = pd.read_excel(file_path, sheet_name='Sheet1')

# Define a city center (approximate coordinates for Barcelona)
city_center_lat, city_center_lon = 41.3851, 2.1734

# Feature Engineering
data['Distance_to_Center'] = np.sqrt((data['Latitude'] - city_center_lat)**2 +
                                     (data['Longitude'] - city_center_lon)**2)
data['Lat_Lon_Product'] = data['Latitude'] * data['Longitude']

# Add clustering (group similar locations)
kmeans = KMeans(n_clusters=3, random_state=42)
data['Cluster'] = kmeans.fit_predict(data[['Latitude', 'Longitude']])

# Features: Latitude, Longitude, Distance to Center, Lat-Lon Product, Cluster
X = data[['Latitude', 'Longitude', 'Distance_to_Center', 'Lat_Lon_Product', 'Cluster']]

# Targets: Temperature, Rainfall, AQI, Humidity
y_temp = data['Avg Annual Temp (°C)']
y_rainfall = data['Avg Annual Rainfall (mm)']
y_aqi = data['Avg Annual AQI']
y_humidity = data['Avg Annual Humidity (%)']

# Split the data into training and testing sets
X_train, X_test, y_temp_train, y_temp_test = train_test_split(X, y_temp, test_size=0.2, random_state=42)
_, _, y_rainfall_train, y_rainfall_test = train_test_split(X, y_rainfall, test_size=0.2, random_state=42)
_, _, y_aqi_train, y_aqi_test = train_test_split(X, y_aqi, test_size=0.2, random_state=42)
_, _, y_humidity_train, y_humidity_test = train_test_split(X, y_humidity, test_size=0.2, random_state=42)

# Train separate Random Forest models for each target variable
rf_temp = RandomForestRegressor(random_state=42)
rf_temp.fit(X_train, y_temp_train)

rf_rainfall = RandomForestRegressor(random_state=42)
rf_rainfall.fit(X_train, y_rainfall_train)

rf_aqi = RandomForestRegressor(random_state=42)
rf_aqi.fit(X_train, y_aqi_train)

rf_humidity = RandomForestRegressor(random_state=42)
rf_humidity.fit(X_train, y_humidity_train)

# Evaluate models on the test set
def evaluate_model(model, X_test, y_test, target_name):
    predictions = model.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    print(f"{target_name} - Mean Squared Error: {mse:.2f}, R2 Score: {r2:.2f}")
    return predictions

print("Model Evaluation:")
evaluate_model(rf_temp, X_test, y_temp_test, "Temperature")
evaluate_model(rf_rainfall, X_test, y_rainfall_test, "Rainfall")
evaluate_model(rf_aqi, X_test, y_aqi_test, "AQI")
evaluate_model(rf_humidity, X_test, y_humidity_test, "Humidity")

# Predict values for new coordinates
def predict_climatic_values(lat, lon):
    distance_to_center = np.sqrt((lat - city_center_lat)**2 + (lon - city_center_lon)**2)
    lat_lon_product = lat * lon
    cluster = kmeans.predict([[lat, lon]])[0]
    coords = np.array([[lat, lon, distance_to_center, lat_lon_product, cluster]])
    temp = rf_temp.predict(coords)[0]
    rainfall = rf_rainfall.predict(coords)[0]
    aqi = rf_aqi.predict(coords)[0]
    humidity = rf_humidity.predict(coords)[0]
    return round(temp, 2), round(rainfall, 2), round(aqi, 2), round(humidity, 2)

# Example: Predict values for a new location
latitude = float(input("Enter Latitude: "))
longitude = float(input("Enter Longitude: "))

temp, rainfall, aqi, humidity = predict_climatic_values(latitude, longitude)
print(f"\nPredicted values at ({latitude}, {longitude}):")
print(f"Temperature: {temp} °C")
print(f"Rainfall: {rainfall} mm")
print(f"AQI: {aqi}")
print(f"Humidity: {humidity} %")
